In [2]:
import os
%pwd

'c:\\Users\\savit\\Research_Summarizer\\research'

In [3]:
os.chdir(r"C:\Users\savit\Research_Summarizer")
%pwd

'C:\\Users\\savit\\Research_Summarizer'

In [4]:
%pwd

'C:\\Users\\savit\\Research_Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    per_device_train_batch_size: int
    per_device_eval_batch_size: int
    evaluation_strategy: str
    eval_steps: int
    save_strategy: str
    save_steps: int
    learning_rate: float
    warmup_steps: int
    weight_decay: float
    logging_steps: int
    fp16: bool
    gradient_accumulation_steps: int
    predict_with_generate: bool
    load_best_model_at_end: bool
    metric_for_best_model: str
    report_to: str


In [6]:
from src.ResearchSummarizer.constants import *
from src.ResearchSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self)  ->  ModelTrainerConfig:
        config=self.config.model_trainer
        params=self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            num_train_epochs= params.num_train_epochs,
            per_device_train_batch_size= params.per_device_train_batch_size,
            per_device_eval_batch_size= params.per_device_eval_batch_size,
            evaluation_strategy= params.evaluation_strategy,
            eval_steps= params.eval_steps,
            save_strategy= params.save_strategy,
            save_steps= params.save_steps,
            learning_rate= params.learning_rate,
            warmup_steps = params.warmup_steps,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            fp16 = params.fp16,
            gradient_accumulation_steps= params.gradient_accumulation_steps,
            predict_with_generate= params.predict_with_generate,
            load_best_model_at_end= params.load_best_model_at_end,
            metric_for_best_model= params.metric_for_best_model,
            report_to = params.report_to,

                
        )
        return model_trainer_config







In [8]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
import torch
from datasets import load_from_disk

c:\Users\savit\Research_Summarizer\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
import evaluate
import numpy as np
from nltk.tokenize import sent_tokenize
import nltk

In [10]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
    
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu" 
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        #loading the data
        dataset_scientific_pt = load_from_disk(self.config.data_path)

        
        training_args = Seq2SeqTrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=3,
            per_device_train_batch_size=4,
            per_device_eval_batch_size=4,
            eval_strategy='epoch',  
            save_strategy='epoch',        
            learning_rate=5e-5,
            warmup_steps=50,
            weight_decay=0.01,
            logging_steps=20,
            fp16=True,
            gradient_accumulation_steps=2,
            predict_with_generate=True,
            load_best_model_at_end=True,
            metric_for_best_model='eval_loss',
            report_to="none"
        )

        rouge_metric = evaluate.load("rouge")

        def compute_metrics(eval_pred):
            predictions, labels = eval_pred
            decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
            labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
            decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
            decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
            result = rouge_metric.compute(
                predictions=decoded_preds,
                references=decoded_labels,
                use_stemmer=True
            )
            result = {key: value * 100 for key, value in result.items()}
            return {k: round(v, 4) for k, v in result.items()}

        trainer = Seq2SeqTrainer(
            model=model_pegasus,
            args=training_args,
            train_dataset=dataset_scientific_pt["train"],
            eval_dataset=dataset_scientific_pt["validation"],
            data_collator=seq2seq_data_collator,
            tokenizer=tokenizer,
            compute_metrics=compute_metrics
        )

        trainer.train()

        model_pegasus.save_trained(os.path.join(self.config.root_dir, "pegasus-scientific-model"))

        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))

        

In [11]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.56.0
Uninstalling transformers-4.56.0:
  Successfully uninstalled transformers-4.56.0
Found existing installation: accelerate 1.10.1
Uninstalling accelerate-1.10.1:
  Successfully uninstalled accelerate-1.10.1
  Using cached transformers-4.56.0-py3-none-any.whl.metadata (40 kB)
  Using cached accelerate-1.10.1-py3-none-any.whl.metadata (19 kB)
Using cached transformers-4.56.0-py3-none-any.whl (11.6 MB)
Using cached accelerate-1.10.1-py3-none-any.whl (374 kB)

   ---------------------------------------- 0/2 [accelerate]
   ---------------------------------------- 0/2 [accelerate]
   ---------------------------------------- 0/2 [accelerate]
   ---------------------------------------- 0/2 [accelerate]
   ---------------------------------------- 0/2 [accelerate]
   ---------------------------------------- 0/2 [accelerate]
   ---------------------------------------- 0/2 [accelerate]
   ---------------------------------------- 0/2 [accelerate]
   -

In [12]:
!pip install huggingface_hub[hf_xet]

In [13]:
config = ConfigurationManager()
model_trainer_config = config.get_model_trainer_config()
model_trainer = ModelTrainer(config=model_trainer_config)
model_trainer.train()


[2025-09-03 12:22:29,205: INFO: common: YAML file loaded successfully: config\config.yaml]
[2025-09-03 12:22:29,214: INFO: common: YAML file loaded successfully: params.yaml]
[2025-09-03 12:22:29,217: INFO: common: ceated dictionary at : artifacts]
[2025-09-03 12:22:29,219: INFO: common: ceated dictionary at : artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\savit\AppData\Local\Temp\ipykernel_22984\618796285.py:51: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.
c:\Users\savit\Research_Summarizer\venv\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found,

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 